In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from copy import deepcopy
import h5py
import numpy as np
import os
import pandas as pd
import scipy.signal as signal
from sklearn.decomposition import PCA
# from spynal.matIO import loadmat
import time
from tqdm.auto import tqdm

from UniversalUnconsciousness.hdf5_utils import *

import h5py
import hydra
from hydra import compose, initialize
import logging
from matplotlib import font_manager
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.decomposition import PCA
import sys
from tqdm.auto import tqdm

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

from UniversalUnconsciousness.data_utils import collect_delase_indices_to_run, collect_grid_indices_to_run, get_grid_params_to_use, get_grid_search_run_list, get_noise_filter_info, get_pca_chosen, get_grid_search_window_ts
from UniversalUnconsciousness.hdf5_utils import convert_char_array, convert_h5_string_array, TransposedDatasetView

In [3]:
with initialize(version_base="1.3", config_path="../UniversalUnconsciousness/DeLASE_analysis/conf"):
    cfg = compose(config_name="config.yaml")

In [4]:
if 'propofol' in cfg.params.data_class:
    session_list = [f[:-4] for f in os.listdir(os.path.join(cfg.params.all_data_dir, 'anesthesia', 'mat', cfg.params.data_class)) if f.endswith('.mat')]
else:
    session_list = [f[:-4] for f in os.listdir(os.path.join(cfg.params.all_data_dir, cfg.params.data_class, 'mat')) if f.endswith('.mat')]
session_list = [session for session in session_list if session not in ['PEDRI_Ketamine_20220203']]

# session_list = [session for session in session_list if 'Dex' not in session]
# session_list = [session for session in session_list if 'Dex' in session]

# areas = ['all']
areas = cfg.params.areas

# session_list = [session_list[0]]

# session_list = session_list[:1]
# log.info(f"Session list: {session_list}")


# --------------------------------------------------------------------------
# Find noisy data
# --------------------------------------------------------------------------
# log.info("-"*20)
# log.info("FINDING NOISY DATA")
# log.info("-"*20)
noise_filter_info = get_noise_filter_info(cfg, session_list, log=None, verbose=True)

# --------------------------------------------------------------------------
# PCA
# --------------------------------------------------------------------------
# log.info("-"*20)
# log.info("RUNNING PCA")
# log.info("-"*20)
pca_chosen = get_pca_chosen(cfg, session_list, areas, noise_filter_info, log=None, verbose=True)

11 bad electrodes, 874 valid windows (out of 915 total windows)
23 bad electrodes, 924 valid windows (out of 1033 total windows)
24 bad electrodes, 948 valid windows (out of 1193 total windows)
22 bad electrodes, 933 valid windows (out of 1396 total windows)
16 bad electrodes, 760 valid windows (out of 1153 total windows)
25 bad electrodes, 1320 valid windows (out of 1516 total windows)
14 bad electrodes, 802 valid windows (out of 873 total windows)
28 bad electrodes, 1068 valid windows (out of 1513 total windows)
21 bad electrodes, 832 valid windows (out of 913 total windows)
13 bad electrodes, 967 valid windows (out of 1033 total windows)
12 bad electrodes, 1301 valid windows (out of 1353 total windows)
22 bad electrodes, 743 valid windows (out of 1276 total windows)
20 bad electrodes, 438 valid windows (out of 913 total windows)
14 bad electrodes, 1317 valid windows (out of 1336 total windows)
19 bad electrodes, 468 valid windows (out of 915 total windows)
11 bad electrodes, 1006 va

In [5]:
session = session_list[0]

In [14]:
# --------------------------------------------------------------------------
# Grid Search
# --------------------------------------------------------------------------

loop_num = 1
while True:
    # log.info(f"GRID SEARCH PROCESS - LOOP #{loop_num}")
    # --------------------------------------------------------------------------
    # Collect indices to run
    # --------------------------------------------------------------------------
    # log.info("-"*20)
    # log.info("COLLECTING INDICES TO RUN")
    # log.info("-"*20)
    
    all_indices_to_run = collect_grid_indices_to_run(cfg, session_list, areas, noise_filter_info, pca_chosen, log=None, verbose=True)

    # --------------------------------------------------------------------------
    # Running grid search
    # --------------------------------------------------------------------------
    if os.path.exists('/om2/user/eisenaj'):
        os.chdir('/om2/user/eisenaj/code/UniversalUnconsciousness/UniversalUnconsciousness')
        batch_size = 25
        # batch_size = 10
    else:
        os.chdir('/home/eisenaj/code/UniversalUnconsciousness/UniversalUnconsciousness')
        batch_size = 25
    # batch_size = cfg.params.batch_size

    if not all_indices_to_run:
        # log.info("No indices to run - breaking")
        break

    sessions_to_run = list(all_indices_to_run.keys())
    # sessions_to_run = sessions_to_run[:4]

    # iterator = tqdm(total=np.sum([len(all_indices_to_run[session]) for session in sessions_to_run]), desc='Hydra Multiprocessing - DSA on Neural Data')
    iterator = tqdm(total=np.sum([np.sum([int(np.ceil(len(all_indices_to_run[session][area])/batch_size)) for area in all_indices_to_run[session].keys()]) for session in sessions_to_run]), desc='Hydra Multiprocessing - Grid Search on Neural Data')

    for session in sessions_to_run:
        for area in all_indices_to_run[session].keys():
            # log.info(f"Running indices for {session} - {area}")
            num_batches = int(np.ceil(len(all_indices_to_run[session][area])/batch_size))
            for batch_num in range(num_batches):
                batch_start = batch_num*batch_size
                batch_end = np.min([(batch_num + 1)*batch_size, len(all_indices_to_run[session][area])])
                # log.info(f"running batch #{batch_num}")
                if cfg.params.pca:
                    os.system(f"HYDRA_FULL_ERROR=1 python DeLASE_analysis/run_grid_search.py -m ++params.session={session} ++params.area={area} ++params.pca_dims={int(pca_chosen[session][area])} ++params.run_index={','.join([str(i) for i in all_indices_to_run[session][area][batch_start:batch_end]])}")
                else:
                    raise ValueError("pca is not True")
                    os.system(f"HYDRA_FULL_ERROR=1 python DeLASE_analysis/run_grid_search.py -m ++params.session={session} ++params.area={area} ++params.run_index={','.join([str(i) for i in all_indices_to_run[session][area][batch_start:batch_end]])}")
                iterator.update()
    iterator.close()

    loop_num += 1

Getting Grid Search Run List:   0%|          | 0/62 [00:00<?, ?it/s]

list exists! loading /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/grid_searches/anesthesiaLvrOdd/grid_search_run_lists/SPOCK_Dexmedetomidine_20211028_grid_set4_window_15...
*COMPLETE*: All results completed for SPOCK_Dexmedetomidine_20211028 - all!!
INCOMPLETE: SPOCK_Dexmedetomidine_20211028 - vlPFC-L incomplete, adding indices [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 

KeyboardInterrupt: 

In [13]:
os.system(f"HYDRA_FULL_ERROR=1 python DeLASE_analysis/run_grid_search.py -m ++params.session={session} ++params.area={area} ++params.run_index={','.join([str(i) for i in all_indices_to_run[session][area][batch_start:batch_end]])}")

[2025-09-03 16:34:51,210][HYDRA] Submitit 'slurm' sweep output dir : multirun/2025-09-03/16-34-48
[2025-09-03 16:34:51,211][HYDRA] 	#0 : ++params.session=SPOCK_Dexmedetomidine_20211028 ++params.area=vlPFC-L ++params.run_index=1
[2025-09-03 16:34:51,215][HYDRA] 	#1 : ++params.session=SPOCK_Dexmedetomidine_20211028 ++params.area=vlPFC-L ++params.run_index=2
[2025-09-03 16:34:51,220][HYDRA] 	#2 : ++params.session=SPOCK_Dexmedetomidine_20211028 ++params.area=vlPFC-L ++params.run_index=3
[2025-09-03 16:34:51,224][HYDRA] 	#3 : ++params.session=SPOCK_Dexmedetomidine_20211028 ++params.area=vlPFC-L ++params.run_index=4
[2025-09-03 16:34:51,227][HYDRA] 	#4 : ++params.session=SPOCK_Dexmedetomidine_20211028 ++params.area=vlPFC-L ++params.run_index=5
[2025-09-03 16:34:51,231][HYDRA] 	#5 : ++params.session=SPOCK_Dexmedetomidine_20211028 ++params.area=vlPFC-L ++params.run_index=6
[2025-09-03 16:34:51,235][HYDRA] 	#6 : ++params.session=SPOCK_Dexmedetomidine_20211028 ++params.area=vlPFC-L ++params.run_

0